In [ ]:
import torch

from models import CocoVGG
from datasets import COCOImgWithLabel, BalancedCocoSuperClassDataset

In [ ]:
class roiVGG(torch.nn.Module):
    def __init__(self, numClasses: int, cocoVGGWeights):
        super(roiVGG, self).__init__()
        self.cocoVgg19 = CocoVGG(numClasses)
        self.cocoVgg19.load_state_dict(cocoVGGWeights)
        for params in self.cocoVgg19.parameters():
            params.requires_grad = False

        self.cocoClassifier = self.cocoVgg19.classifier
        self.roiClassifier = 


    def forward(self):


In [ ]:
class VGGROI (torch.nn.Module):
    def __init__(self, numROIs):
        super(VGGROI, self).__init__()
        self.vgg = vgg19(weights="DEFAULT")
        self.convFeatures = self.vgg.features[:36]
        self.maxPool = torch.nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0, dilation = 1, ceil_mode = False)
        self.adaptiveAvgPool = torch.nn.AdaptiveAvgPool2d(output_size = (7, 7))
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(in_features = 25088, out_features = 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(in_features = 4096, out_features = 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(in_features = 1024, out_features = numROIs)
        )
        # self.restOfModel = torch.nn.Sequential(
        #     self.maxPool,
        #     self.adaptiveAvgPool,
        #     self.classifier
        # )
        self.gradients = None
    def activations_hook(self, grad):
        self.gradients = grad
    def forward(self, img):
        extractedFeatures = self.convFeatures(img)
        hook = extractedFeatures.register_hook(self.activations_hook)
        rest = self.maxPool(extractedFeatures)
        rest = self.adaptiveAvgPool(rest)
        rest = torch.flatten(rest, 1)
        rest = self.classifier(rest)
        return rest
    def get_activation_gradient(self):
        return self.gradients
    def get_activations(self, img):
        return self.convFeatures(img)